In [36]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
import scikit_posthocs as sp



In [81]:
path = '/export/usuarios_ml4ds/ammesa/Data/4_indexed_data/res/ZS_toy_dir'
df_weighted = pd.read_csv(os.path.join(path, 'modifica.csv'), sep=';')
df_re = pd.read_parquet(os.path.join(path, 'full_metrics.parquet'))

In [82]:
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI
0,0,toy_questions,TB_ENN,mrr@3,"0,233333333","0,245326449"
1,1,toy_questions,TB_ENN,mrr@5,"0,200833333","0,146790239"
2,2,toy_questions,TB_ENN,mrr@10,"0,166203704","0,099721803"
3,3,toy_questions,TB_ENN,precision@3,"0,1","0,094676995"
4,4,toy_questions,TB_ENN,precision@5,"0,14","0,096816858"
...,...,...,...,...,...,...
223,223,toy_questions,ANN,hit@10,"0,2","0,247922569"
224,224,toy_questions,ANN,rank_hit@3,0,0
225,225,toy_questions,ANN,rank_hit@5,0,0
226,226,toy_questions,ANN,rank_hit@10,"0,0225","0,028105676"


In [83]:
df_re

,mrr_3,precision_3,recall_3,ndcg_3,hit_3,rank_hit_3,mrr_5,precision_5,recall_5,ndcg_5,...,mrr_10,precision_10,recall_10,ndcg_10,hit_10,rank_hit_10,time,retrieval_method,thr,weighted
0,0.000000,0.000000,0.00,0.00000,0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.237881,TB_ENN,0,True
1,0.500000,0.333333,0.50,0.63093,1,0.500000,0.500000,0.2,0.500000,0.630930,...,0.300000,0.2,1.00,0.564092,1,0.500000,0.232176,TB_ENN,0,True
2,0.333333,0.333333,0.25,0.50000,1,0.333333,0.333333,0.2,0.250000,0.500000,...,0.208333,0.3,0.75,0.549841,1,0.333333,0.215198,TB_ENN,0,True
3,1.000000,0.333333,0.20,1.00000,0,0.000000,1.000000,0.2,0.200000,1.000000,...,0.414286,0.3,0.60,0.761357,0,0.000000,0.214440,TB_ENN,0,True
4,0.000000,0.000000,0.00,0.00000,0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.227475,TB_ENN,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,1.000000,0.333333,0.20,1.00000,0,0.000000,1.000000,0.2,0.200000,1.000000,...,0.562500,0.2,0.40,0.806574,0,0.000000,0.009180,ANN,1,True
356,0.000000,0.000000,0.00,0.00000,0,0.000000,0.250000,0.2,0.333333,0.430677,...,0.164286,0.3,1.00,0.494186,1,0.100000,0.009047,ANN,1,True
357,0.000000,0.000000,0.00,0.00000,0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.125000,0.1,1.00,0.315465,1,0.125000,0.009251,ANN,1,True
358,0.000000,0.000000,0.00,0.00000,0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.009259,ANN,1,True


In [84]:
def get_significance_marker( p_value):
    if p_value < 0.001:
        return "***"
    elif p_value < 0.01:
        return "**"
    elif p_value < 0.05:
        return "*"
    else:
        return ""
    
def format_metric(metric):
    return (
            "MRR@3" if metric == "mrr_3" else
            "MRR@5" if metric == "mrr_5" else
            "NDCG@3" if metric == "ndcg_3" else
            "NDCG@5" if metric == "ndcg_5" else
            "Precision@3" if metric == "precision_3" else
            "Precision@5" if metric == "precision_5" else
            "Recall@3" if metric == "recall_3" else
            "Recall@5" if metric == "recall_5" else
            "RHit@3" if metric == "rank_hit_3" else
            "RHit@5" if metric == "rank_hit_5" else
            "Time (s)" if "time" in metric else
            metric  # Fallback si no está en la lista
        )
        

def check_normality( values):
    stat, p_value = stats.shapiro(values)
    return p_value > 0.05  

def perform_anova( groups):
    f_stat, p_value = stats.f_oneway(*groups)
    return f_stat, p_value

def perform_kruskal_wallis( groups):
    h_stat, p_value = stats.kruskal(*groups)
    return h_stat, p_value

# Normality tests:

In [85]:
df_re

,mrr_3,precision_3,recall_3,ndcg_3,hit_3,rank_hit_3,mrr_5,precision_5,recall_5,ndcg_5,...,mrr_10,precision_10,recall_10,ndcg_10,hit_10,rank_hit_10,time,retrieval_method,thr,weighted
0,0.000000,0.000000,0.00,0.00000,0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.237881,TB_ENN,0,True
1,0.500000,0.333333,0.50,0.63093,1,0.500000,0.500000,0.2,0.500000,0.630930,...,0.300000,0.2,1.00,0.564092,1,0.500000,0.232176,TB_ENN,0,True
2,0.333333,0.333333,0.25,0.50000,1,0.333333,0.333333,0.2,0.250000,0.500000,...,0.208333,0.3,0.75,0.549841,1,0.333333,0.215198,TB_ENN,0,True
3,1.000000,0.333333,0.20,1.00000,0,0.000000,1.000000,0.2,0.200000,1.000000,...,0.414286,0.3,0.60,0.761357,0,0.000000,0.214440,TB_ENN,0,True
4,0.000000,0.000000,0.00,0.00000,0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.227475,TB_ENN,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,1.000000,0.333333,0.20,1.00000,0,0.000000,1.000000,0.2,0.200000,1.000000,...,0.562500,0.2,0.40,0.806574,0,0.000000,0.009180,ANN,1,True
356,0.000000,0.000000,0.00,0.00000,0,0.000000,0.250000,0.2,0.333333,0.430677,...,0.164286,0.3,1.00,0.494186,1,0.100000,0.009047,ANN,1,True
357,0.000000,0.000000,0.00,0.00000,0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.125000,0.1,1.00,0.315465,1,0.125000,0.009251,ANN,1,True
358,0.000000,0.000000,0.00,0.00000,0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.009259,ANN,1,True


In [86]:
    
df_weighted["BestValue"] = df_weighted.groupby(["FileID", "Metric"])["Weighted Mean"].transform(
    lambda x: x.min() if "Time (s)" in x.name else x.max()
)
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,0,toy_questions,TB_ENN,mrr@3,"0,233333333","0,245326449","0,233333333"
1,1,toy_questions,TB_ENN,mrr@5,"0,200833333","0,146790239","0,200833333"
2,2,toy_questions,TB_ENN,mrr@10,"0,166203704","0,099721803","0,175185185"
3,3,toy_questions,TB_ENN,precision@3,"0,1","0,094676995","0,133333333"
4,4,toy_questions,TB_ENN,precision@5,"0,14","0,096816858","0,16"
...,...,...,...,...,...,...,...
223,223,toy_questions,ANN,hit@10,"0,2","0,247922569","0,5"
224,224,toy_questions,ANN,rank_hit@3,0,0,"0,133333333"
225,225,toy_questions,ANN,rank_hit@5,0,0,"0,133333333"
226,226,toy_questions,ANN,rank_hit@10,"0,0225","0,028105676","0,168333333"


In [87]:
df_weighted = df_weighted[~df_weighted['Metric'].isin(['hit@3', 'hit@5','hit@10','rank_hit@5', 'rank_hit@3','rank_hit@10' ])]
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,0,toy_questions,TB_ENN,mrr@3,"0,233333333","0,245326449","0,233333333"
1,1,toy_questions,TB_ENN,mrr@5,"0,200833333","0,146790239","0,200833333"
2,2,toy_questions,TB_ENN,mrr@10,"0,166203704","0,099721803","0,175185185"
3,3,toy_questions,TB_ENN,precision@3,"0,1","0,094676995","0,133333333"
4,4,toy_questions,TB_ENN,precision@5,"0,14","0,096816858","0,16"
...,...,...,...,...,...,...,...
217,217,toy_questions,ANN,recall@10,"0,24","0,246679841","0,52"
218,218,toy_questions,ANN,ndcg@3,"0,1","0,185941926","0,25"
219,219,toy_questions,ANN,ndcg@5,"0,143067656","0,194109157","0,305333415"
220,220,toy_questions,ANN,ndcg@10,"0,161622404","0,167815426","0,379285369"


In [88]:
df_re['Method'] = df_re['retrieval_method'].astype(str)

# Add '_W' if weighted is True
df_re['Method'] += df_re['weighted'].apply(lambda x: '-W' if x else '')

# Add '_V' if thr == 1
df_re['Method'] += df_re['thr'].apply(lambda x: '-V' if x == 1 else '')


In [89]:
df_re = df_re.drop(columns=['hit_3', 'hit_5', 'hit_10',
                            'rank_hit_3', 'rank_hit_5', 'rank_hit_10',])

In [90]:
df_re

,mrr_3,precision_3,recall_3,ndcg_3,mrr_5,precision_5,recall_5,ndcg_5,mrr_10,precision_10,recall_10,ndcg_10,time,retrieval_method,thr,weighted,Method
0,0.000000,0.000000,0.00,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.237881,TB_ENN,0,True,TB_ENN-W
1,0.500000,0.333333,0.50,0.63093,0.500000,0.2,0.500000,0.630930,0.300000,0.2,1.00,0.564092,0.232176,TB_ENN,0,True,TB_ENN-W
2,0.333333,0.333333,0.25,0.50000,0.333333,0.2,0.250000,0.500000,0.208333,0.3,0.75,0.549841,0.215198,TB_ENN,0,True,TB_ENN-W
3,1.000000,0.333333,0.20,1.00000,1.000000,0.2,0.200000,1.000000,0.414286,0.3,0.60,0.761357,0.214440,TB_ENN,0,True,TB_ENN-W
4,0.000000,0.000000,0.00,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.227475,TB_ENN,0,True,TB_ENN-W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,1.000000,0.333333,0.20,1.00000,1.000000,0.2,0.200000,1.000000,0.562500,0.2,0.40,0.806574,0.009180,ANN,1,True,ANN-W-V
356,0.000000,0.000000,0.00,0.00000,0.250000,0.2,0.333333,0.430677,0.164286,0.3,1.00,0.494186,0.009047,ANN,1,True,ANN-W-V
357,0.000000,0.000000,0.00,0.00000,0.000000,0.0,0.000000,0.000000,0.125000,0.1,1.00,0.315465,0.009251,ANN,1,True,ANN-W-V
358,0.000000,0.000000,0.00,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.009259,ANN,1,True,ANN-W-V


In [91]:
# Step 2: Drop the old method and modifiers if you don't need them
df_re = df_re.drop(columns=['retrieval_method', 'thr', 'weighted'])

# Step 3: Melt the dataframe so that metric names become a column, and values are aligned accordingly
df_melted = df_re.melt(id_vars=['Method'], var_name='Metric', value_name='Value')

df_melted

,Method,Metric,Value
0,TB_ENN-W,mrr_3,0.000000
1,TB_ENN-W,mrr_3,0.500000
2,TB_ENN-W,mrr_3,0.333333
3,TB_ENN-W,mrr_3,1.000000
4,TB_ENN-W,mrr_3,0.000000
...,...,...,...
4675,ANN-W-V,time,0.009180
4676,ANN-W-V,time,0.009047
4677,ANN-W-V,time,0.009251
4678,ANN-W-V,time,0.009259


In [92]:
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,0,toy_questions,TB_ENN,mrr@3,"0,233333333","0,245326449","0,233333333"
1,1,toy_questions,TB_ENN,mrr@5,"0,200833333","0,146790239","0,200833333"
2,2,toy_questions,TB_ENN,mrr@10,"0,166203704","0,099721803","0,175185185"
3,3,toy_questions,TB_ENN,precision@3,"0,1","0,094676995","0,133333333"
4,4,toy_questions,TB_ENN,precision@5,"0,14","0,096816858","0,16"
...,...,...,...,...,...,...,...
217,217,toy_questions,ANN,recall@10,"0,24","0,246679841","0,52"
218,218,toy_questions,ANN,ndcg@3,"0,1","0,185941926","0,25"
219,219,toy_questions,ANN,ndcg@5,"0,143067656","0,194109157","0,305333415"
220,220,toy_questions,ANN,ndcg@10,"0,161622404","0,167815426","0,379285369"


In [93]:
# Replace commas with dots and convert last 3 columns to float
df_weighted.iloc[:, -3:] = df_weighted.iloc[:, -3:].apply(
    lambda col: pd.to_numeric(col.str.replace(',', '.', regex=False), errors='coerce')
)
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,0,toy_questions,TB_ENN,mrr@3,0.233333,0.245326,0.233333
1,1,toy_questions,TB_ENN,mrr@5,0.200833,0.14679,0.200833
2,2,toy_questions,TB_ENN,mrr@10,0.166204,0.099722,0.175185
3,3,toy_questions,TB_ENN,precision@3,0.1,0.094677,0.133333
4,4,toy_questions,TB_ENN,precision@5,0.14,0.096817,0.16
...,...,...,...,...,...,...,...
217,217,toy_questions,ANN,recall@10,0.24,0.24668,0.52
218,218,toy_questions,ANN,ndcg@3,0.1,0.185942,0.25
219,219,toy_questions,ANN,ndcg@5,0.143068,0.194109,0.305333
220,220,toy_questions,ANN,ndcg@10,0.161622,0.167815,0.379285


In [95]:
def format_weighted_value(row):
    mean = f"{row['Weighted Mean']:.3f}"
    ci = f"{row['95% CI']:.3f}"
    value = f"{mean} \pm {ci}"

    metric = row["Metric"]
    method = row["Method"]

    """
    # Agregar '*' si Kruskal-Wallis indica diferencias significativas
    if metric in significance_dict["Kruskal-Wallis p-value"]:
        kw_p_value = significance_dict["Kruskal-Wallis p-value"][metric]
        if kw_p_value < 0.05:
            value += " *"  # Indica que hay diferencias globales
    """
    significance_marker = ""
    if metric in significance_dict["Significant post-hoc tests (Dunn)"]:
        dunn_df = significance_dict["Significant post-hoc tests (Dunn)"][metric]
        for other_method in dunn_df.index:
            if method in dunn_df.columns and other_method in dunn_df.index:
                p_val = dunn_df.loc[other_method, method]
                if p_val < 0.001:
                    significance_marker = "***"
                elif p_val < 0.01:
                    significance_marker = "**"
                elif p_val < 0.05:
                    significance_marker = "*"

        value += f"^{{{significance_marker}}}"
    
    if row["Weighted Mean"] == row["BestValue"]:
        return f"\\(\\boldsymbol{{{value}}}\\)"
    else:
        return f"\\({value}\\)"

df_long = df_melted
df_weighted['Weighted Mean'] = pd.to_numeric(df_weighted['Weighted Mean'], errors='coerce')
df_weighted['95% CI'] = pd.to_numeric(df_weighted['95% CI'], errors='coerce')
normality_tests = {}
print(df_weighted)
for metric in df_long["Metric"].unique():
    for method in df_long["Method"].unique():
        values = df_long[(df_long["Metric"] == metric) & (df_long["Method"] == method)]["Value"]
        values = df_long[
            (df_long["Metric"] == metric) & (df_long["Method"] == method)
        ]["Value"]

        # Clean and convert to float
        values = pd.to_numeric(values, errors="coerce").dropna()
        if len(values) > 3:  # Shapiro necesita al menos unos pocos valores
            stat, p_value = stats.shapiro(values)
            normality_tests[(metric, method)]=  p_value

# ANOVA or Kruskal-Wallis
anova_results = {}
kruskal_results = {}

for metric in df_long["Metric"].unique():
    groups = [df_long[(df_long["Metric"] == metric) & (df_long["Method"] == method)]["Value"].dropna() for method in df_long["Method"].unique()]
    groups = [pd.to_numeric(g, errors="coerce").dropna() for g in groups]
    # if normality in at least one group
    normal = all(p > 0.05 for p in [normality_tests.get((metric, method), 0) for method in df_long["Method"].unique()])
    
    if normal:
        stat, p_value = stats.f_oneway(*groups)
        anova_results[metric] = p_value
    else:
        stat, p_value = stats.kruskal(*groups)
        kruskal_results[metric] = p_value

# post-hoc tests if Kruskal-Wallis is significant
posthoc_results = {}
for metric, p in kruskal_results.items():
    if p < 0.05:  # Si hay diferencias significativas
        df_filtered = df_long[df_long["Metric"] == metric]
        posthoc = sp.posthoc_dunn(df_filtered, val_col="Value", group_col="Method", p_adjust="bonferroni")
        posthoc_results[metric] = posthoc


    significance_dict = {
        "ANOVA p-value": anova_results,
        "Kruskal-Wallis p-value": kruskal_results,
        "Significant post-hoc tests (Dunn)": posthoc_results
    }
    df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
    
    df_weighted_pivot = df_weighted.pivot_table(index=["FileID", "Method"], columns="Metric", values="Weighted Mean ± CI", aggfunc="first")
    
    latex_output = "\\begin{table*}[h]\n\\centering\n\\resizebox{\\textwidth}{!}{%\n\\begin{tabular}{lc" + "c" * len(df_weighted_pivot.columns) + "}\n\\arrayrulecolor{black}\n\\toprule\n"
    
    latex_output += "File & \\textbf{Method} & " + " & ".join([f"\\textbf{{{col}}}" for col in df_weighted_pivot.columns]) + " \\\\\n\\midrule\n"
    
    current_file_id = None 
    for (file_idx, method), row in df_weighted_pivot.iterrows():
        if file_idx != current_file_id:  
            if current_file_id is not None:
                latex_output += "\\arrayrulecolor{black}\\specialrule{0.5pt}{0pt}{0pt}\\arrayrulecolor{black}\n"

            col_n = str(len(df_weighted_pivot.columns) + 2)
            file_id = str(file_idx+'1')
            
            latex_output += f"\\multicolumn{{{col_n}}}{{l}}{{\\textbf{{File {file_id}}}}} \\\\\n"
            
            latex_output += "\\arrayrulecolor{black}\\specialrule{0.5pt}{0pt}{0pt}\\arrayrulecolor{black}\n"
            
            current_file_id = file_idx  
        
        row_start = "\\rowcolor{tableblue} " if method == "XX" else ""
        
        latex_output += row_start + f"& \\textbf{{{method}}} & " + " & ".join(row.astype(str)) + " \\\\\n"
    
    latex_output += "\\bottomrule\n\\end{tabular}}\n\\caption{Performance Metrics per Method for Each File (Best values in bold)}\n\\label{tab:grouped_results}\n\\end{table*}"
    
    print(latex_output)


<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_2965158/146575848.py:4: SyntaxWarning: invalid escape sequence '\p'
  value = f"{mean} \pm {ci}"
/tmp/ipykernel_2965158/146575848.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted['Weighted Mean'] = pd.to_numeric(df_weighted['Weighted Mean'], errors='coerce')
/tmp/ipykernel_2965158/146575848.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted['95% CI'] = pd.to_numeri

     Unnamed: 0         FileID  Method       Metric  Weighted Mean    95% CI  \
0             0  toy_questions  TB_ENN        mrr@3       0.233333  0.245326   
1             1  toy_questions  TB_ENN        mrr@5       0.200833  0.146790   
2             2  toy_questions  TB_ENN       mrr@10       0.166204  0.099722   
3             3  toy_questions  TB_ENN  precision@3       0.100000  0.094677   
4             4  toy_questions  TB_ENN  precision@5       0.140000  0.096817   
..          ...            ...     ...          ...            ...       ...   
217         217  toy_questions     ANN    recall@10       0.240000  0.246680   
218         218  toy_questions     ANN       ndcg@3       0.100000  0.185942   
219         219  toy_questions     ANN       ndcg@5       0.143068  0.194109   
220         220  toy_questions     ANN      ndcg@10       0.161622  0.167815   
227         227  toy_questions     ANN         time       0.009190  0.000065   

    BestValue                Weighted M

/tmp/ipykernel_2965158/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
/tmp/ipykernel_2965158/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
/tmp/ipykernel_2965158/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

\begin{table*}[h]
\centering
\resizebox{\textwidth}{!}{%
\begin{tabular}{lcccccccccccccc}
\arrayrulecolor{black}
\toprule
File & \textbf{Method} & \textbf{mrr@10} & \textbf{mrr@3} & \textbf{mrr@5} & \textbf{ndcg@10} & \textbf{ndcg@3} & \textbf{ndcg@5} & \textbf{precision@10} & \textbf{precision@3} & \textbf{precision@5} & \textbf{recall@10} & \textbf{recall@3} & \textbf{recall@5} & \textbf{time} \\
\midrule
\multicolumn{15}{l}{\textbf{File toy_questions1}} \\
\arrayrulecolor{black}\specialrule{0.5pt}{0pt}{0pt}\arrayrulecolor{black}
& \textbf{ANN} & \(0.085 \pm 0.105\) & \(0.100 \pm 0.186\) & \(0.125 \pm 0.187\) & \(0.162 \pm 0.168\) & \(0.100 \pm 0.186\) & \(0.143 \pm 0.194\) & \(0.060 \pm 0.063\) & \(0.033 \pm 0.062\) & \(0.040 \pm 0.050\) & \(0.240 \pm 0.247\) & \(0.020 \pm 0.037\) & \(0.053 \pm 0.069\) & \(0.009 \pm 0.000\) \\
& \textbf{ENN} & \(\boldsymbol{0.175 \pm 0.104}\) & \(0.200 \pm 0.211\) & \(0.189 \pm 0.135\) & \(\boldsymbol{0.379 \pm 0.187}\) & \(0.242 \pm 0.241\) & \(\bo

/tmp/ipykernel_2965158/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
/tmp/ipykernel_2965158/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
/tmp/ipykernel_2965158/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [34]:
significance_dict['Significant post-hoc tests (Dunn)']

{'mrr_3':                    ANN-W       ANN-W-V          ENN-W        ENN-W-V  \
 ANN-W       1.000000e+00  1.000000e+00   3.240123e-62   3.240123e-62   
 ANN-W-V     1.000000e+00  1.000000e+00   3.232833e-62   3.232833e-62   
 ENN-W       3.240123e-62  3.232833e-62   1.000000e+00   1.000000e+00   
 ENN-W-V     3.240123e-62  3.232833e-62   1.000000e+00   1.000000e+00   
 TB_ANN      5.361214e-64  5.348981e-64   1.000000e+00   1.000000e+00   
 TB_ANN-V    8.349842e-18  8.360023e-18  9.697983e-147  9.697983e-147   
 TB_ANN-W    8.782529e-12  8.773743e-12   1.498992e-19   1.498992e-19   
 TB_ANN-W-V  5.519311e-18  5.526073e-18  3.004911e-147  3.004911e-147   
 TB_ENN      3.466505e-63  3.458645e-63   1.000000e+00   1.000000e+00   
 TB_ENN-V    1.363826e-03  1.364636e-03   1.729176e-97   1.729176e-97   
 TB_ENN-W    6.792231e-89  6.773983e-89   6.320562e-02   6.320562e-02   
 TB_ENN-W-V  2.573891e-03  2.575374e-03   3.643197e-96   3.643197e-96   
 
                    TB_ANN       TB_ANN-

In [35]:
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue,Weighted Mean ± CI
0,0,micro_questions_sports,TB_ENN,mrr@3,NaN,NaN,"0,470428762",\(nan \pm nan\)
1,1,micro_questions_sports,TB_ENN,mrr@5,NaN,NaN,"0,482240574",\(nan \pm nan\)
2,2,micro_questions_sports,TB_ENN,mrr@10,NaN,NaN,"0,492548699",\(nan \pm nan\)
3,3,micro_questions_sports,TB_ENN,precision@3,NaN,NaN,"0,182349016",\(nan \pm nan\)
4,4,micro_questions_sports,TB_ENN,precision@5,NaN,NaN,"0,11961962",\(nan \pm nan\)
...,...,...,...,...,...,...,...,...
217,217,micro_questions_sports,ANN,recall@10,NaN,NaN,"0,675925926",\(nan \pm nan\)
218,218,micro_questions_sports,ANN,ndcg@3,NaN,NaN,"0,490019367",\(nan \pm nan\)
219,219,micro_questions_sports,ANN,ndcg@5,NaN,NaN,"0,511172373",\(nan \pm nan\)
220,220,micro_questions_sports,ANN,ndcg@10,NaN,NaN,"0,536405059",\(nan \pm nan\)
